In [3]:
from pathlib import Path
import fireducks.pandas as pd

In [4]:
# Validar esquemas de archivos parquet en un directorio
def validar_esquemas(directorio: Path) -> None:
    """
    Valida que todos los archivos parquet tengan el mismo esquema
    Imprime resultados y no retorna nada
    """
    archivos_parquet = list(directorio.glob('*.parquet'))
    
    if not archivos_parquet:
        print(f"❌ No se encontraron archivos parquet en '{directorio}'")
        return None
    
    print(f"📁 Archivos encontrados: {[p.name for p in archivos_parquet]}")
    print(f"📊 Total archivos: {len(archivos_parquet)}\n")
    
    # Leer esquemas de cada archivo
    esquemas = {}
    shapes = {}
    
    for archivo in archivos_parquet:
        df = pd.read_parquet(archivo)
        esquema = df.dtypes.to_dict()
        esquemas[archivo.name] = esquema
        shapes[archivo.name] = df.shape
        print(f"🔍 {archivo.name}: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    
    # Comparar esquemas
    print("\n" + "="*50)
    print("VALIDACIÓN DE ESQUEMAS")
    print("="*50)
    
    # Esquema de referencia (primer archivo)
    archivo_referencia = archivos_parquet[0].name
    esquema_referencia = esquemas[archivo_referencia]
    
    esquemas_iguales = True
    diferencias = []
    
    for nombre_archivo, esquema in esquemas.items():
        if esquema != esquema_referencia:
            esquemas_iguales = False
            diferencias.append(nombre_archivo)
            print(f"❌ {nombre_archivo} tiene esquema diferente")
            
            # Mostrar diferencias específicas
            cols_ref = set(esquema_referencia.keys())
            cols_actual = set(esquema.keys())
            
            # Columnas faltantes
            if cols_ref - cols_actual:
                print(f"   📉 Columnas faltantes: {cols_ref - cols_actual}")
            
            # Columnas extras
            if cols_actual - cols_ref:
                print(f"   📈 Columnas extras: {cols_actual - cols_ref}")
            
            # Tipos diferentes
            for col in cols_ref & cols_actual:
                if esquema_referencia[col] != esquema[col]:
                    print(f"   🔄 '{col}': {esquema_referencia[col]} → {esquema[col]}")
        else:
            print(f"✅ {nombre_archivo} esquema coincide")
    
    if esquemas_iguales:
        print(f"\n🎉 TODOS LOS ESQUEMAS SON IDÉNTICOS")
        print(f"📋 Esquema común ({len(esquema_referencia)} columnas):")
        for col, tipo in esquema_referencia.items():
            print(f"   • {col}: {tipo}")
    else:
        print(f"\n⚠️  ESQUEMAS DIFERENTES DETECTADOS")
        print(f"📋 Archivos con diferencias: {diferencias}")
    
    return None


# Ejecutar validación
data_path = Path('../data/raw')
validar_esquemas(data_path)

📁 Archivos encontrados: ['part-00000-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2513-1-c000.snappy.parquet', 'part-00001-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2514-1-c000.snappy.parquet', 'part-00002-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2515-1-c000.snappy.parquet', 'part-00003-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2516-1-c000.snappy.parquet', 'part-00004-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2517-1-c000.snappy.parquet', 'part-00005-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2518-1-c000.snappy.parquet', 'part-00006-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2519-1-c000.snappy.parquet', 'part-00007-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2520-1-c000.snappy.parquet']
📊 Total archivos: 8

🔍 part-00000-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2513-1-c000.snappy.parquet: 155,648 filas × 14 columnas
🔍 part-0

In [5]:
def ingesta_archivos(directorio: Path) -> pd.DataFrame:
    """
    Carga y concatena (apila) todos los archivos parquet del directorio.
    Los datos crecen a nivel de registros (filas), no de columnas.
    """
    archivos_parquet = list(directorio.glob('*.parquet'))
    
    if not archivos_parquet:
        print(f"❌ No se encontraron archivos parquet en '{directorio}'")
        return pd.DataFrame()
    
    print(f"📁 Archivos encontrados: {[p.name for p in archivos_parquet]}\n")
    
    # Cargar todos los dataframes
    dfs = []
    filas_total = 0
    
    for archivo in archivos_parquet:
        df = pd.read_parquet(archivo)
        df = df.drop_duplicates()  # Eliminar duplicados internos
        dfs.append(df)
        filas_total += len(df)
        print(f"{archivo.name}: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    
    print("\n" + "="*50)
    print("CONCATENANDO ARCHIVOS")
    print("="*50)
    
    # Concatenar todos los dataframes
    df_final = pd.concat(dfs, ignore_index=True)
    
    print(f"Total esperado: {filas_total:,} filas")
    print(f"Dataset final: {df_final.shape[0]:,} filas × {df_final.shape[1]} columnas")
    
    # Validaciones
    if len(df_final) == filas_total:
        print("✅ Concatenación exitosa - todas las filas conservadas")
    else:
        print("⚠️ ALERTA: Se perdieron filas en la concatenación")
    
    # Validar duplicados por cliente_id
    clientes_unicos = df_final['cliente_id'].nunique()
    total_filas = len(df_final)
    
    print(f"\nClientes únicos: {clientes_unicos:,}")
    print(f"Total registros: {total_filas:,}")
    print(f"Promedio registros por cliente: {total_filas/clientes_unicos:.1f}")
    
    return df_final

# Ejecutar ingesta
df_unidos = ingesta_archivos(data_path)

📁 Archivos encontrados: ['part-00000-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2513-1-c000.snappy.parquet', 'part-00001-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2514-1-c000.snappy.parquet', 'part-00002-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2515-1-c000.snappy.parquet', 'part-00003-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2516-1-c000.snappy.parquet', 'part-00004-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2517-1-c000.snappy.parquet', 'part-00005-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2518-1-c000.snappy.parquet', 'part-00006-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2519-1-c000.snappy.parquet', 'part-00007-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2520-1-c000.snappy.parquet']

part-00000-tid-4846308366011599585-72d1a41b-9ea0-4c9f-95c6-325ac28a2af6-2513-1-c000.snappy.parquet: 155,648 filas × 14 columnas
part-00001-tid-484630836601159

In [6]:
# Distribución de canal_pedido_cd
df_unidos['canal_pedido_cd'].value_counts(normalize=True)

canal_pedido_cd
DIGITAL     0.479390
VENDEDOR    0.335487
TELEFONO    0.185122
Name: proportion, dtype: float64

In [7]:
# Muestra de clientes
sample = df_unidos['cliente_id'].sample(n=5, random_state=777).to_list()

for id in sample:
    display(df_unidos[df_unidos['cliente_id'] == id].sort_values(by='fecha_pedido_dt'))

,cliente_id,pais_cd,region_comercial_txt,agencia_id,ruta_id,tipo_cliente_cd,madurez_digital_cd,estrellas_txt,frecuencia_visitas_cd,fecha_pedido_dt,canal_pedido_cd,facturacion_usd_val,materiales_distintos_val,cajas_fisicas
272425,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2023-01-26,DIGITAL,115.20,14,8.67
132264,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2023-02-21,DIGITAL,99.10,8,12.85
1217980,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2023-02-27,DIGITAL,173.50,1,15.22
1182661,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2023-03-20,DIGITAL,45.62,14,3.20
455022,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2023-05-29,DIGITAL,105.20,11,12.32
1139038,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2024-03-11,VENDEDOR,81.26,6,13.57
11035,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2024-03-25,DIGITAL,165.90,4,13.66
316737,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2024-07-08,VENDEDOR,37.02,2,19.60
638118,C024423,EC,AMAZONIA,EC-AMA-A004,EC-AMA-A004-R002,TIENDA,MEDIA,2,LMV,2024-07-23,DIGITAL,140.39,5,17.78


,cliente_id,pais_cd,region_comercial_txt,agencia_id,ruta_id,tipo_cliente_cd,madurez_digital_cd,estrellas_txt,frecuencia_visitas_cd,fecha_pedido_dt,canal_pedido_cd,facturacion_usd_val,materiales_distintos_val,cajas_fisicas
533105,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2023-07-27,VENDEDOR,236.02,14,19.01
822297,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2023-10-03,VENDEDOR,152.92,2,11.99
24702,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2023-10-27,VENDEDOR,74.79,1,8.84
80433,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2024-01-17,VENDEDOR,128.02,6,6.77
963769,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2024-06-01,VENDEDOR,93.81,11,14.75
270628,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2024-06-03,VENDEDOR,134.33,10,8.64
868793,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2024-07-23,VENDEDOR,52.01,4,11.85
1228976,C040122,EC,SIERRA,EC-SIE-A005,EC-SIE-A005-R017,TIENDA,BAJA,1,LMV,2024-08-14,TELEFONO,169.82,14,8.73


,cliente_id,pais_cd,region_comercial_txt,agencia_id,ruta_id,tipo_cliente_cd,madurez_digital_cd,estrellas_txt,frecuencia_visitas_cd,fecha_pedido_dt,canal_pedido_cd,facturacion_usd_val,materiales_distintos_val,cajas_fisicas
775290,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-02-22,DIGITAL,100.40,6,10.82
811201,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-06-08,VENDEDOR,184.95,5,13.72
880429,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-07-09,DIGITAL,81.77,13,26.49
1233587,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-09-24,DIGITAL,98.14,7,11.79
376444,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-10-10,DIGITAL,112.27,8,11.98
909769,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2023-12-30,TELEFONO,211.08,12,9.90
1236611,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2024-01-09,DIGITAL,172.34,11,13.29
1115606,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2024-01-20,DIGITAL,86.71,12,11.01
106607,C125720,GT,SUR,GT-SUR-A002,GT-SUR-A002-R010,TIENDA,MEDIA,3,L,2024-05-07,VENDEDOR,76.03,3,13.02


,cliente_id,pais_cd,region_comercial_txt,agencia_id,ruta_id,tipo_cliente_cd,madurez_digital_cd,estrellas_txt,frecuencia_visitas_cd,fecha_pedido_dt,canal_pedido_cd,facturacion_usd_val,materiales_distintos_val,cajas_fisicas
847321,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-02-17,DIGITAL,157.48,8,10.74
1185161,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-02-25,TELEFONO,170.40,9,7.86
318789,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-03-26,TELEFONO,129.52,14,13.78
130586,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-04-22,VENDEDOR,83.04,7,18.78
42656,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-06-07,DIGITAL,156.69,13,15.37
1135505,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-12-21,DIGITAL,68.70,4,17.33
707360,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-12-23,VENDEDOR,113.30,4,16.19
762925,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2023-12-25,DIGITAL,52.76,8,17.78
1225771,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2024-01-10,TELEFONO,109.88,6,14.62
436153,C025883,EC,SIERRA,EC-SIE-A003,EC-SIE-A003-R011,MINIMARKET,MEDIA,3,LMI,2024-07-13,TELEFONO,45.61,13,15.14


,cliente_id,pais_cd,region_comercial_txt,agencia_id,ruta_id,tipo_cliente_cd,madurez_digital_cd,estrellas_txt,frecuencia_visitas_cd,fecha_pedido_dt,canal_pedido_cd,facturacion_usd_val,materiales_distintos_val,cajas_fisicas
456899,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2023-02-27,TELEFONO,111.53,2,18.53
880109,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2023-04-27,DIGITAL,107.37,10,13.96
444156,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2023-06-02,VENDEDOR,78.04,14,11.40
861196,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2023-10-11,VENDEDOR,48.68,12,13.24
854141,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2024-01-21,DIGITAL,171.09,6,1.74
481204,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2024-03-11,DIGITAL,205.68,14,4.99
282615,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2024-04-09,VENDEDOR,167.45,4,8.91
933494,C115225,SV,CENTRO,SV-CEN-A002,SV-CEN-A002-R004,TIENDA,BAJA,1,LMV,2024-06-22,DIGITAL,117.59,1,11.74


In [8]:
# Convertir estrellas_txt a numérico
df_unidos['estrellas_txt'] = pd.to_numeric(df_unidos['estrellas_txt'], errors='coerce')

# Definir agregaciones por categoría
agregaciones = {
    # Información demográfica (invariante por cliente)
    'pais_cd': 'first',
    'region_comercial_txt': 'first',
    'agencia_id': 'first',
    'ruta_id': 'first',
    'tipo_cliente_cd': 'first',
    'madurez_digital_cd': 'first',
    'estrellas_txt': 'first',
    'frecuencia_visitas_cd': 'first',
    
    # Métricas de negocio (sumatoria)
    'facturacion_usd_val': 'sum',
    'materiales_distintos_val': 'sum',
    'cajas_fisicas': 'sum',
    
    # Información temporal y comportamental
    'fecha_pedido_dt': 'max',  # Último pedido
    'canal_pedido_cd': 'last'  # Último canal usado
}

# Crear dataset agregado por cliente
df_agregado = (
    df_unidos
    .groupby('cliente_id')
    .agg(agregaciones)
    .reset_index()
)

# Mostrar estado inicial
print(f"Datos iniciales: {len(df_agregado):,} filas")
print(f"Clientes únicos iniciales: {df_agregado['cliente_id'].nunique():,}")

# Eliminar duplicados completos
df_agregado = df_agregado.drop_duplicates()
print(f"Después de eliminar duplicados exactos: {len(df_agregado):,} filas")

# Eliminar duplicados por cliente_id (mantener último)
df_agregado = df_agregado.drop_duplicates(subset=['cliente_id'], keep='last')
print(f"Después de eliminar duplicados por cliente_id: {len(df_agregado):,} filas")

# Verificar resultado final
print(f"Clientes únicos finales: {df_agregado['cliente_id'].nunique():,}")

Datos iniciales: 149,960 filas
Clientes únicos iniciales: 149,960
Después de eliminar duplicados exactos: 149,960 filas
Después de eliminar duplicados por cliente_id: 149,960 filas
Clientes únicos finales: 149,960


In [9]:
# Distribución de canal_pedido_cd
df_agregado['canal_pedido_cd'].value_counts(normalize=True)

canal_pedido_cd
DIGITAL     0.478368
VENDEDOR    0.336930
TELEFONO    0.184703
Name: proportion, dtype: float64

In [19]:
# Identificar columnas categóricas excluyendo IDs
categorical_columns = df_agregado.select_dtypes(include=['object', 'category']).columns.tolist()

# Filtrar columnas que son IDs
ids_to_exclude = [col for col in df_agregado.columns if '_id' in col]
categorical_columns = [col for col in categorical_columns if col not in ids_to_exclude]

print(f"Total: {len(categorical_columns)} variables")
print("-" * 40)

# Mostrar etiquetas únicas para cada variable categórica
for col in categorical_columns:
    print(f"Variable: {col}")
    print(df_agregado[col].value_counts())
    print("-" * 40)

Total: 6 variables
----------------------------------------
Variable: pais_cd
pais_cd
GT    37640
EC    37586
PE    37443
SV    37291
Name: count, dtype: int64
----------------------------------------
Variable: region_comercial_txt
region_comercial_txt
OCCIDENTE        33403
SIERRA           17301
NORTE            15537
CENTRO           14806
SUR              12258
AMAZONIA         11939
METROPOLITANA    11300
SELVA            10262
LIMA              9161
COSTA             8346
ORIENTE           5647
Name: count, dtype: int64
----------------------------------------
Variable: tipo_cliente_cd
tipo_cliente_cd
TIENDA        89937
MINIMARKET    44998
MAYORISTA     15025
Name: count, dtype: int64
----------------------------------------
Variable: madurez_digital_cd
madurez_digital_cd
BAJA     75024
MEDIA    52546
ALTA     22390
Name: count, dtype: int64
----------------------------------------
Variable: frecuencia_visitas_cd
frecuencia_visitas_cd
LM     37674
L      37540
LMI    37390
LMV  